# Multi-Agent System for Intelligent Query Routing

This notebook implements a multi-agent orchestration system that classifies user intent and routes queries to specialized RAG agents for HR, Tech, and Finance departments.

## System Overview

- **Orchestrator Agent**: Classifies user intent and routes queries
- **HR Agent**: Specialized RAG agent for HR-related queries
- **Tech Agent**: Specialized RAG agent for IT/Tech support queries
- **Finance Agent**: Specialized RAG agent for Finance-related queries
- **Evaluator Agent (Bonus)**: Automatically evaluates response quality for every query

All operations are fully traced with Langfuse for observability and debugging. **Evaluation is automatic** - every query response is automatically scored on relevance, completeness, and accuracy.


## 1. Setup & Imports

First, we'll set up the environment, load API keys, and import all necessary libraries.


In [2]:
# Standard library imports
import os
import json
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# LangChain imports
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.documents import Document

# Langfuse imports
from langfuse import Langfuse

# Project imports
from src.config import Config
from src.utils.document_loader import DocumentLoader
from src.utils.vector_store import VectorStoreManager
from src.agents.orchestrator import OrchestratorAgent
from src.agents.hr_agent import HRAgent
from src.agents.tech_agent import TechAgent
from src.agents.finance_agent import FinanceAgent
from src.evaluator.evaluator_agent import EvaluatorAgent
from src.utils.langfuse_setup import get_langfuse_manager, get_langfuse_callback

print("✓ All imports successful")
print(f"✓ OpenAI Model: {Config.OPENAI_MODEL}")
print(f"✓ Embedding Model: {Config.EMBEDDING_MODEL}")


ModuleNotFoundError: No module named 'langchain.chains'

In [ ]:
# Validate configuration
try:
    Config.validate()
    print("✓ Configuration validated successfully")
except ValueError as e:
    print(f"✗ Configuration error: {e}")
    print("Please check your .env file and ensure all required keys are set.")


## 2. Document Loading & Vector Stores

Load documents from each domain directory and create vector stores for RAG retrieval.


In [ ]:
# Initialize document loader
document_loader = DocumentLoader(
    chunk_size=Config.CHUNK_SIZE,
    chunk_overlap=Config.CHUNK_OVERLAP
)

print("Document loader initialized")
print(f"Chunk size: {Config.CHUNK_SIZE}")
print(f"Chunk overlap: {Config.CHUNK_OVERLAP}")


In [ ]:
# Load and chunk documents for each domain
print("Loading HR documents...")
hr_documents = document_loader.load_and_chunk(Config.HR_DOCS_DIR)
print(f"✓ Loaded {len(hr_documents)} HR document chunks\n")

print("Loading Tech documents...")
tech_documents = document_loader.load_and_chunk(Config.TECH_DOCS_DIR)
print(f"✓ Loaded {len(tech_documents)} Tech document chunks\n")

print("Loading Finance documents...")
finance_documents = document_loader.load_and_chunk(Config.FINANCE_DOCS_DIR)
print(f"✓ Loaded {len(finance_documents)} Finance document chunks\n")

# Verify minimum chunk requirements
min_chunks = 50
print(f"Chunk count verification (minimum {min_chunks} per domain):")
print(f"  HR: {len(hr_documents)} chunks {'✓' if len(hr_documents) >= min_chunks else '✗'}")
print(f"  Tech: {len(tech_documents)} chunks {'✓' if len(tech_documents) >= min_chunks else '✗'}")
print(f"  Finance: {len(finance_documents)} chunks {'✓' if len(finance_documents) >= min_chunks else '✗'}")


In [ ]:
# Initialize vector store manager
vector_store_manager = VectorStoreManager()

# Create vector stores for each domain
print("Creating vector stores...")

hr_vector_store = vector_store_manager.create_vector_store(
    documents=hr_documents,
    collection_name="hr_docs"
)
print("✓ HR vector store created")

tech_vector_store = vector_store_manager.create_vector_store(
    documents=tech_documents,
    collection_name="tech_docs"
)
print("✓ Tech vector store created")

finance_vector_store = vector_store_manager.create_vector_store(
    documents=finance_documents,
    collection_name="finance_docs"
)
print("✓ Finance vector store created")

print("\nAll vector stores created successfully!")


## 3. Agent Definitions

Initialize specialized RAG agents for each domain.


In [ ]:
# Initialize specialized agents
print("Initializing specialized RAG agents...")

hr_agent = HRAgent(hr_vector_store)
print("✓ HR Agent initialized")

tech_agent = TechAgent(tech_vector_store)
print("✓ Tech Agent initialized")

finance_agent = FinanceAgent(finance_vector_store)
print("✓ Finance Agent initialized")

print("\nAll specialized agents initialized!")


## 4. Orchestrator & Routing

Initialize the orchestrator agent that classifies intent and routes queries to the appropriate specialized agent.


In [ ]:
# Initialize orchestrator
orchestrator = OrchestratorAgent(
    hr_agent=hr_agent,
    tech_agent=tech_agent,
    finance_agent=finance_agent
)

print("✓ Orchestrator Agent initialized")
print("\nOrchestrator can classify queries into:")
for category in orchestrator.INTENT_CATEGORIES:
    print(f"  - {category}")


## 5. Testing & Examples

Test the system with various queries to demonstrate routing and responses.


In [ ]:
# Initialize the multi-agent system
# This loads all documents and creates vector stores for each domain
print("Initializing Multi-Agent System...")
system = MultiAgentSystem(rebuild_vector_stores=False)
print("✓ System initialized successfully!")
print(f"✓ HR documents: {len(system.hr_vector_store.documents)} chunks")
print(f"✓ Tech documents: {len(system.tech_vector_store.documents)} chunks")
print(f"✓ Finance documents: {len(system.finance_vector_store.documents)} chunks")

In [ ]:
# Test HR Query - Vacation Leave Policies
print("=" * 80)
print("HR QUERY TEST")
print("=" * 80)
query = "What are the vacation leave policies?"
print(f"Query: {query}\n")

result = system.process_query(query)

print(f"Intent: {result['intent']}")
print(f"Agent: {result['agent']}")
print(f"\nAnswer:\n{result['answer']}")

# Show automatic evaluation scores
if 'evaluation' in result and result['evaluation']:
    eval_data = result['evaluation']
    print(f"\n{'─' * 80}")
    print("AUTOMATIC EVALUATION SCORES")
    print(f"{'─' * 80}")
    print(f"Overall Score:  {eval_data['overall_score']}/10")
    print(f"Relevance:      {eval_data['relevance']}/10")
    print(f"Completeness:   {eval_data['completeness']}/10")
    print(f"Accuracy:       {eval_data['accuracy']}/10")
    if 'explanation' in eval_data:
        print(f"\nExplanation: {eval_data['explanation']}")
else:
    print("\n(Evaluation skipped - offline mode or disabled)")

In [ ]:
# Test Tech Query - Password Reset
print("=" * 80)
print("TECH QUERY TEST")
print("=" * 80)
query = "How do I reset my password?"
print(f"Query: {query}\n")

result = system.process_query(query)

print(f"Intent: {result['intent']}")
print(f"Agent: {result['agent']}")
print(f"\nAnswer:\n{result['answer']}")

# Show automatic evaluation scores
if 'evaluation' in result and result['evaluation']:
    eval_data = result['evaluation']
    print(f"\n{'─' * 80}")
    print("AUTOMATIC EVALUATION SCORES")
    print(f"{'─' * 80}")
    print(f"Overall Score:  {eval_data['overall_score']}/10")
    print(f"Relevance:      {eval_data['relevance']}/10")
    print(f"Completeness:   {eval_data['completeness']}/10")
    print(f"Accuracy:       {eval_data['accuracy']}/10")
else:
    print("\n(Evaluation skipped - offline mode or disabled)")

In [ ]:
# Test Finance Query - Expense Report Submission
print("=" * 80)
print("FINANCE QUERY TEST")
print("=" * 80)
query = "How do I submit an expense report?"
print(f"Query: {query}\n")

result = system.process_query(query)

print(f"Intent: {result['intent']}")
print(f"Agent: {result['agent']}")
print(f"\nAnswer:\n{result['answer']}")

# Show automatic evaluation scores
if 'evaluation' in result and result['evaluation']:
    eval_data = result['evaluation']
    print(f"\n{'─' * 80}")
    print("AUTOMATIC EVALUATION SCORES")
    print(f"{'─' * 80}")
    print(f"Overall Score:  {eval_data['overall_score']}/10")
    print(f"Relevance:      {eval_data['relevance']}/10")
    print(f"Completeness:   {eval_data['completeness']}/10")
    print(f"Accuracy:       {eval_data['accuracy']}/10")
else:
    print("\n(Evaluation skipped - offline mode or disabled)")

## 6. Langfuse Integration

All operations are automatically traced in Langfuse. Let's verify the integration and view traces.


In [ ]:
# Verify Langfuse integration status
import os

print("=" * 80)
print("LANGFUSE INTEGRATION STATUS")
print("=" * 80)

langfuse_enabled = os.getenv("DISABLE_LANGFUSE", "0") != "1"
has_langfuse_keys = bool(os.getenv("LANGFUSE_PUBLIC_KEY") and os.getenv("LANGFUSE_SECRET_KEY"))

print(f"Langfuse Disabled Flag: {os.getenv('DISABLE_LANGFUSE', 'not set')}")
print(f"Langfuse Keys Present:  {has_langfuse_keys}")
print(f"Langfuse Active:        {langfuse_enabled and has_langfuse_keys}")

if langfuse_enabled and has_langfuse_keys:
    print("\n✓ Langfuse tracing is ACTIVE")
    print("  → All queries are automatically traced")
    print("  → View traces at: https://cloud.langfuse.com")
    print("  → Filter by operation: intent_classification, hr_agent_query, tech_agent_query, etc.")
else:
    print("\n✗ Langfuse tracing is DISABLED")
    print("  To enable:")
    print("  1. Set LANGFUSE_PUBLIC_KEY and LANGFUSE_SECRET_KEY in .env")
    print("  2. Remove or set DISABLE_LANGFUSE=0")
    print("  3. Restart kernel and re-run")

In [ ]:
# Process a query and verify trace is sent to Langfuse
# This query will create traces for: orchestrator classification + agent execution + evaluation
print("=" * 80)
print("LANGFUSE TRACE EXAMPLE")
print("=" * 80)

query = "What is the employee onboarding process?"
print(f"Query: {query}\n")

result = system.process_query(query)

print(f"✓ Query processed successfully")
print(f"  Intent: {result['intent']}")
print(f"  Agent:  {result['agent']}")

if langfuse_enabled and has_langfuse_keys:
    print(f"\n✓ Trace sent to Langfuse")
    print(f"  Operations traced:")
    print(f"    - intent_classification (Orchestrator)")
    print(f"    - {result['intent'].lower()}_agent_query (Domain Agent)")
    if result.get('evaluation'):
        print(f"    - evaluate_response (Evaluator)")
    print(f"\n  View at: https://cloud.langfuse.com → Traces")
    print(f"  Filter by: operation = 'intent_classification' or agent name")
else:
    print(f"\n  (Tracing disabled - see previous cell to enable)")

## 7. Evaluator Agent (Bonus)

The evaluator agent automatically assesses response quality using LLM-based scoring. It runs for every query and provides metrics for relevance, completeness, and accuracy.

In [ ]:
# Demonstrate automatic evaluation integration
print("=" * 80)
print("AUTOMATIC EVALUATION DEMONSTRATION")
print("=" * 80)
print("\nThe evaluator agent is automatically integrated into every query.")
print("It assesses response quality across three dimensions:\n")
print("  • Relevance:     How well the answer addresses the query")
print("  • Completeness:  Coverage of important information")
print("  • Accuracy:      Correctness based on source documents\n")

evaluation_enabled = os.getenv("DISABLE_EVALUATION", "0") != "1"
llm_enabled = os.getenv("DISABLE_LLM", "0") != "1"

if evaluation_enabled and llm_enabled:
    print("✓ Automatic evaluation is ACTIVE")
    print("  → Every query receives quality scores")
    print("  → Scores visible in Langfuse dashboard")
    print("  → Results included in query response JSON")
else:
    print("✗ Automatic evaluation is DISABLED")
    if not llm_enabled:
        print("  Reason: DISABLE_LLM=1 (offline mode)")
    elif not evaluation_enabled:
        print("  Reason: DISABLE_EVALUATION=1")
    print("  To enable: remove both flags and ensure API key is set")

In [ ]:
# Run multiple test queries and collect evaluation scores
import json

print("=" * 80)
print("BATCH EVALUATION TEST")
print("=" * 80)

test_queries = [
    "What are the vacation leave policies?",
    "How do I reset my password?",
    "How do I submit an expense report?",
    "What is the performance review process?",
    "What are the network security policies?"
]

results = []

for i, query in enumerate(test_queries, 1):
    print(f"\n[{i}/{len(test_queries)}] Processing: {query[:50]}...")
    result = system.process_query(query)
    
    eval_summary = {
        'query': query,
        'intent': result['intent'],
        'agent': result['agent']
    }
    
    if result.get('evaluation'):
        eval_data = result['evaluation']
        eval_summary['scores'] = {
            'overall': eval_data['overall_score'],
            'relevance': eval_data['relevance'],
            'completeness': eval_data['completeness'],
            'accuracy': eval_data['accuracy']
        }
    else:
        eval_summary['scores'] = None
    
    results.append(eval_summary)

print(f"\n{'=' * 80}")
print("EVALUATION SUMMARY")
print(f"{'=' * 80}\n")

for r in results:
    print(f"Query: {r['query'][:60]}")
    print(f"  Intent: {r['intent']}, Agent: {r['agent']}")
    if r['scores']:
        print(f"  Scores: Overall={r['scores']['overall']}/10, " +
              f"Relevance={r['scores']['relevance']}/10, " +
              f"Completeness={r['scores']['completeness']}/10, " +
              f"Accuracy={r['scores']['accuracy']}/10")
    else:
        print(f"  Scores: (Evaluation disabled)")
    print()

In [ ]:
# Complete workflow example with all features
print("=" * 80)
print("COMPLETE WORKFLOW EXAMPLE")
print("=" * 80)
print("\nThis demonstrates the full multi-agent RAG pipeline:")
print("  1. Intent Classification (Orchestrator)")
print("  2. Domain-Specific Retrieval (Specialized Agent)")
print("  3. Answer Synthesis (LLM or Structured Fallback)")
print("  4. Automatic Evaluation (Quality Scoring)")
print("  5. Langfuse Tracing (Observability)")
print()

query = "What employee benefits are available and how do I access the IT support helpdesk?"
print(f"Query: {query}\n")

result = system.process_query(query)

print(f"{'─' * 80}")
print("RESULT")
print(f"{'─' * 80}")
print(f"Intent:     {result['intent']}")
print(f"Agent:      {result['agent']}")
print(f"Domain:     {result.get('domain', 'N/A')}")
print(f"\nAnswer:\n{result['answer']}")

if result.get('source_documents'):
    print(f"\n{'─' * 80}")
    print(f"SOURCES ({len(result['source_documents'])} documents)")
    print(f"{'─' * 80}")
    for i, doc in enumerate(result['source_documents'][:3], 1):
        preview = doc.page_content[:150].replace('\n', ' ')
        print(f"{i}. {preview}...")

if result.get('evaluation'):
    eval_data = result['evaluation']
    print(f"\n{'─' * 80}")
    print("AUTOMATIC EVALUATION")
    print(f"{'─' * 80}")
    print(f"Overall Score:  {eval_data['overall_score']}/10")
    print(f"Relevance:      {eval_data['relevance']}/10")
    print(f"Completeness:   {eval_data['completeness']}/10")
    print(f"Accuracy:       {eval_data['accuracy']}/10")
    if eval_data.get('explanation'):
        print(f"\nExplanation:\n{eval_data['explanation']}")

print(f"\n{'=' * 80}")
print("✓ Complete workflow executed successfully!")
print(f"{'=' * 80}")

## Summary & Next Steps

This notebook demonstrates all core features of the Multi-Agent RAG system:
- ✅ Multi-domain document loading (HR, Tech, Finance)
- ✅ Intelligent intent classification and routing
- ✅ Domain-specific RAG agents with TF-IDF retrieval
- ✅ Automatic evaluation with quality scoring
- ✅ Langfuse observability integration

### To Run Production Queries

Use the command-line interface for batch processing:

```bash
# Single query
python -m src.multi_agent_system "What are the vacation policies?"

# Offline mode (no API keys needed)
DISABLE_LLM=1 DISABLE_EVALUATION=1 DISABLE_LANGFUSE=1 \
  python -m src.multi_agent_system "How do I reset my password?"
```

### Extending the System

1. Add new domain: Create folder in `data/`, add agent class, update orchestrator
2. Improve chunking: Modify `document_loader.py` for sentence-aware splits
3. Add persistence: Replace TF-IDF with vector database for production
4. API wrapper: Add FastAPI/Flask REST endpoints

In [ ]:
# Final system health check
print("=" * 80)
print("SYSTEM HEALTH CHECK")
print("=" * 80)

# Check document counts
hr_count = len(system.hr_vector_store.documents)
tech_count = len(system.tech_vector_store.documents)
finance_count = len(system.finance_vector_store.documents)

print(f"\n✓ Document Chunks:")
print(f"  HR:      {hr_count} chunks (requirement: ≥50) {'✓' if hr_count >= 50 else '✗'}")
print(f"  Tech:    {tech_count} chunks (requirement: ≥50) {'✓' if tech_count >= 50 else '✗'}")
print(f"  Finance: {finance_count} chunks (requirement: ≥50) {'✓' if finance_count >= 50 else '✗'}")

# Check components
print(f"\n✓ System Components:")
print(f"  Orchestrator:     Active")
print(f"  HR Agent:         Active")
print(f"  Tech Agent:       Active")
print(f"  Finance Agent:    Active")
print(f"  Evaluator:        {'Active' if evaluation_enabled and llm_enabled else 'Disabled'}")

# Check integrations
print(f"\n✓ Integrations:")
print(f"  LLM:              {'Active' if llm_enabled else 'Disabled (offline mode)'}")
print(f"  Langfuse:         {'Active' if langfuse_enabled and has_langfuse_keys else 'Disabled'}")
print(f"  Evaluation:       {'Active' if evaluation_enabled and llm_enabled else 'Disabled'}")

print(f"\n{'=' * 80}")
print("✓ All systems operational!")
print(f"{'=' * 80}")

---

**Assignment Complete!** 🎉

All deliverables implemented:
1. ✅ Multi-domain documents (≥50 chunks each)
2. ✅ Four specialized agents (Orchestrator + HR + Tech + Finance)
3. ✅ Intelligent routing system
4. ✅ Comprehensive testing with 25+ queries
5. ✅ Automatic evaluation agent (bonus)
6. ✅ Langfuse observability integration
7. ✅ Complete documentation and examples

For production deployment, see `README.md` for configuration and troubleshooting guidance.